#### IR 3

##### Code and Output:

In [4]:
import numpy as np
import pandas as pd
from pgmpy.models import DiscreteBayesianNetwork
from pgmpy.estimators import MaximumLikelihoodEstimator
from pgmpy.inference import VariableElimination

# Read Cleveland Heart Disease data (replace '?' automatically)
heart_data = pd.read_csv('heart.csv', na_values='?')

# Rename target column if needed
if 'target' in heart_data.columns:
    heart_data.rename(columns={'target': 'heartdisease'}, inplace=True)

print("Dataset Loaded Successfully!\n")
print("Sample Data:\n", heart_data.head())


Dataset Loaded Successfully!

Sample Data:
    age  sex  cp  trestbps  chol  fbs  restecg  thalach  exang  oldpeak  slope  \
0   52    1   0       125   212    0        1      168      0      1.0      2   
1   53    1   0       140   203    1        0      155      1      3.1      0   
2   70    1   0       145   174    0        1      125      1      2.6      0   
3   61    1   0       148   203    0        1      161      0      0.0      2   
4   62    0   0       138   294    1        1      106      0      1.9      1   

   ca  thal  heartdisease  
0   2     3             0  
1   0     3             0  
2   0     3             0  
3   1     3             0  
4   3     2             0  


In [6]:
# -----------------------------
# Step 2: Define Bayesian Network Structure
# -----------------------------
# Define the dependency structure based on medical knowledge
model = DiscreteBayesianNetwork([
    ('age', 'trestbps'),          # Age affects resting blood pressure
    ('age', 'fbs'),               # Age affects fasting blood sugar
    ('sex', 'trestbps'),          # Sex affects blood pressure
    ('exang', 'trestbps'),        # Exercise-induced angina affects blood pressure
    ('trestbps', 'heartdisease'), # Blood pressure influences heart disease
    ('fbs', 'heartdisease'),      # Fasting blood sugar influences heart disease
    ('heartdisease', 'restecg'),  # Heart disease affects ECG readings
    ('heartdisease', 'thalach'),  # Heart disease affects maximum heart rate
    ('heartdisease', 'chol')      # Heart disease affects cholesterol
])

print("\n Bayesian Network Structure Defined.")


 Bayesian Network Structure Defined.


In [7]:
# -----------------------------
# Step 3: Train Model using MLE
# -----------------------------
print("\n Training the Bayesian Network...")
model.fit(heart_data, estimator=MaximumLikelihoodEstimator)
print(" Model Training Completed!")

INFO:pgmpy: Datatype (N=numerical, C=Categorical Unordered, O=Categorical Ordered) inferred from data: 
 {'age': 'N', 'sex': 'N', 'cp': 'N', 'trestbps': 'N', 'chol': 'N', 'fbs': 'N', 'restecg': 'N', 'thalach': 'N', 'exang': 'N', 'oldpeak': 'N', 'slope': 'N', 'ca': 'N', 'thal': 'N', 'heartdisease': 'N'}



 Training the Bayesian Network...
 Model Training Completed!


In [8]:
# -----------------------------
# Step 4: Perform Inference
# -----------------------------
print("\n Performing Inference using Variable Elimination...")
HeartDisease_infer = VariableElimination(model)

# Example 1: Probability of Heart Disease given Age
print("\n Probability of Heart Disease given Age = 45:")
q1 = HeartDisease_infer.query(variables=['heartdisease'], evidence={'age': 45})
print(q1)

# Example 2: Probability of Heart Disease given Cholesterol level
print("\n Probability of Heart Disease given Cholesterol = 200:")
q2 = HeartDisease_infer.query(variables=['heartdisease'], evidence={'chol': 200})
print(q2)

# Example 3: Probability of Heart Disease given Multiple Evidence
print("\n Probability of Heart Disease given Age = 50, Sex = 1 (Male), Chol = 240:")
q3 = HeartDisease_infer.query(
    variables=['heartdisease'],
    evidence={'age': 50, 'sex': 1, 'chol': 240}
)
print(q3)

print("\n Heart Disease Diagnosis Complete!")


 Performing Inference using Variable Elimination...

 Probability of Heart Disease given Age = 45:
+-----------------+---------------------+
| heartdisease    |   phi(heartdisease) |
+=================+=====================+
| heartdisease(0) |              0.3169 |
+-----------------+---------------------+
| heartdisease(1) |              0.6831 |
+-----------------+---------------------+

 Probability of Heart Disease given Cholesterol = 200:
+-----------------+---------------------+
| heartdisease    |   phi(heartdisease) |
+=================+=====================+
| heartdisease(0) |              1.0000 |
+-----------------+---------------------+
| heartdisease(1) |              0.0000 |
+-----------------+---------------------+

 Probability of Heart Disease given Age = 50, Sex = 1 (Male), Chol = 240:
+-----------------+---------------------+
| heartdisease    |   phi(heartdisease) |
+=================+=====================+
| heartdisease(0) |              0.0000 |
+------------